# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import numpy as np
import pandas as pd
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report

from sqlalchemy import create_engine

[nltk_data] Downloading package punkt to /Users/zacks/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/zacks/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/zacks/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [1]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need
# 2) run the cells below to read dataset
# 3) follow the steps below to find the answer to the quiz question
from pyspark.ml import Pipeline
from pyspark.ml.feature import CountVectorizer, IDF, Normalizer, PCA, RegexTokenizer, StandardScaler, StopWordsRemover, StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [2]:
import os

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local") \
    .appName("Disaster Response Pipeline") \
    .config("spark.jars", "sqlite-jdbc-3.36.0.3.jar") \
    .getOrCreate()

22/01/19 13:23:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
# org.sqlite.JDBC
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars file:///sqlite-jdbc-3.36.0.3.jar'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.postgresql:postgresql:42.1.1 pyspark-shell'

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.config("spark.jars", "/usr/share/java/mysql-connector-java-8.0.22.jar") \
    .master("local").appName("PySpark_MySQL_test").getOrCreate()

wine_df = spark.read.format("jdbc").option("url", "jdbc:mysql://localhost:3306/TestDB") \
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "Wines") \
    .option("user", "me").option("password", "me").load()

In [11]:
driver = "org.sqlite.JDBC"
# driver = "sqlite-jdbc-3.36.0.3.jar"
path = 'disaster_response.db'
url = "jdbc:sqlite:///" + path
# url = "jdbc:sqlite:///disaster_response.db"
tablename = 'disaster_response'

In [12]:
df = spark.read.format("jdbc").option("url", url)\
.option("dbtable", tablename).option("driver", driver).load()
df.show()

Py4JJavaError: An error occurred while calling o51.load.
: java.sql.SQLException: opening db: '///disaster_response.db': Read-only file system
	at org.sqlite.SQLiteConnection.open(SQLiteConnection.java:231)
	at org.sqlite.SQLiteConnection.<init>(SQLiteConnection.java:61)
	at org.sqlite.jdbc3.JDBC3Connection.<init>(JDBC3Connection.java:28)
	at org.sqlite.jdbc4.JDBC4Connection.<init>(JDBC4Connection.java:21)
	at org.sqlite.JDBC.createConnection(JDBC.java:115)
	at org.sqlite.JDBC.connect(JDBC.java:90)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.BasicConnectionProvider.getConnection(BasicConnectionProvider.scala:49)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.ConnectionProvider$.create(ConnectionProvider.scala:68)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.$anonfun$createConnectionFactory$1(JdbcUtils.scala:64)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$.getQueryOutputSchema(JDBCRDD.scala:62)
	at org.apache.spark.sql

In [13]:
spark.read.jdbc?

Signature:
spark.read.jdbc(
    url,
    table,
    column=None,
    lowerBound=None,
    upperBound=None,
    numPartitions=None,
    predicates=None,
    properties=None,
)
Docstring:
Construct a :class:`DataFrame` representing the database table named ``table``
accessible via JDBC URL ``url`` and connection ``properties``.

Partitions of the table will be retrieved in parallel if either ``column`` or
``predicates`` is specified. ``lowerBound``, ``upperBound`` and ``numPartitions``
is needed when ``column`` is specified.

If both ``column`` and ``predicates`` are specified, ``column`` will be used.

.. versionadded:: 1.4.0

Parameters
----------
table : str
    the name of the table
column : str, optional
    alias of ``partitionColumn`` option. Refer to ``partitionColumn`` in
    `Data Source Option <https://spark.apache.org/docs/latest/sql-data-sources-jdbc.html#data-source-option>`_
    in the version you use.
predicates : list, optional
    a list of expressions suitable for incl

In [ ]:
df = spark.read.table

In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql('select * from disaster_response', con=engine)
X = df.message
Y = df.loc[:, 'related':]
target_names = Y.columns
y = Y.to_numpy()

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(message, stem='lemm'):
    """Text processing.
    
    Args:
        stem(str): stem or lemm.
        
    Returns:
        list: Cleaned tokens.
    """
    # 1. Cleaning
    
    # 2. Normalization
    text = re.sub(r"[^a-zA-Z0-9]", " ", message.lower())
    
    # 3. Tokenization
    tokens = word_tokenize(text)
    
    # 4. Stop Word Removal
    stop_words = stopwords.words("english")
    tokens = list(filter(lambda w: w not in stop_words, tokens))
    
    # 5. Part of Speech Tagging / Named Entity Recognition
    
    # 6. Stemming or Lemmatization
    # Because the targets are not roots, we should use Lemmatization
    
    clean_tokens = []
    if stem == 'stem':
        stemmer = PorterStemmer()
        for tok in tokens:
            clean_tok = stemmer.stem(tok).strip()
            clean_tokens.append(clean_tok)
    else:
        lemmatizer = WordNetLemmatizer()
        for tok in tokens:
            clean_tok = lemmatizer.lemmatize(tok, pos='v').strip()
            clean_tokens.append(clean_tok)

    return clean_tokens

In [4]:
# Test
# eat - starving, water - thirsty should be the same meanings.
# Therefore, the ML models should build the correct word embeddings.
print('Message:', X[10])
print('Targes:', Y.columns.to_list())

Message: There's nothing to eat and water, we starving and thirsty.
Targes: ['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [5]:
print('Stemming', tokenize(X[10], stem='stem'))
print('Lemmatization', tokenize(X[10], stem='lemm'))

Stemming ['noth', 'eat', 'water', 'starv', 'thirsti']
Lemmatization ['nothing', 'eat', 'water', 'starving', 'thirsty']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
# Neural Network needs much more time for modeling
# mlp = MLPClassifier(random_state=42, max_iter=200, verbose=True, early_stopping=True)
forest = RandomForestClassifier(random_state=42)

pipeline = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenize)),
    ('multi_clf', MultiOutputClassifier(forest))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

In [9]:
# Baseline score
pipeline.score(X_test, y_test)

0.2679252479023646

In [10]:
for i in range(len(target_names)):
    print(target_names[i])
    print(classification_report(y_test[:, i], y_pred[:, i]))
    print('-'*60)

related
              precision    recall  f1-score   support

           0       0.71      0.42      0.53      1266
           1       0.83      0.94      0.88      3938
           2       0.39      0.45      0.42        40

    accuracy                           0.81      5244
   macro avg       0.64      0.60      0.61      5244
weighted avg       0.80      0.81      0.79      5244

------------------------------------------------------------
request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4349
           1       0.83      0.48      0.61       895

    accuracy                           0.89      5244
   macro avg       0.87      0.73      0.77      5244
weighted avg       0.89      0.89      0.88      5244

------------------------------------------------------------
offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      

/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classificat

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5177
           1       0.00      0.00      0.00        67

    accuracy                           0.99      5244
   macro avg       0.49      0.50      0.50      5244
weighted avg       0.97      0.99      0.98      5244

------------------------------------------------------------
other_infrastructure
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      5021
           1       0.00      0.00      0.00       223

    accuracy                           0.96      5244
   macro avg       0.48      0.50      0.49      5244
weighted avg       0.92      0.96      0.94      5244

------------------------------------------------------------
weather_related
              precision    recall  f1-score   support

           0       0.89      0.95      0.92      3806
           1       0.84      0.69      0.76      1438

    accuracy           

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   TfidfVectorizer(tokenizer=<function tokenize at 0x136ac3700>)),
  ('multi_clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42)))],
 'verbose': False,
 'vect': TfidfVectorizer(tokenizer=<function tokenize at 0x136ac3700>),
 'multi_clf': MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42)),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.float64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__norm': 'l2',
 'vect__preprocessor': None,
 'vect__smooth_idf': True,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__sublinear_tf': False,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(message, stem='lemm')>,
 'vect__use_idf': True,
 'vect__vocabulary':

In [15]:
def build_model():
    # Neural Network needs much more time for modeling
    # mlp = MLPClassifier(random_state=42, max_iter=200, verbose=True, early_stopping=True)
    forest = RandomForestClassifier(random_state=42)

    pipeline = Pipeline([
        ('vect', TfidfVectorizer(tokenizer=tokenize)),
        ('multi_clf', MultiOutputClassifier(forest))
    ])

    # specify parameters for grid search
    parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.5, 0.75, 1.0),
        'vect__max_features': (None, 5000, 10000),
        'vect__smooth_idf': (True, False),
        'vect__use_idf': (True, False),
        'multi_clf__estimator__max_depth': (None, 300, 500),
        'multi_clf__estimator__n_estimators': [50, 100, 200, 300],
        'multi_clf__estimator__min_samples_split': [2, 3, 4]
    }

    # create grid search object
    cv = GridSearchCV(pipeline, param_grid=parameters, scoring='accuracy', verbose=0)
    
    return cv

In [ ]:
model = build_model()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
print("Best Parameters:", model.best_params_)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.